Original Code (as of 5/13/23)

In [56]:
#Import the needed modules
import klayout.db as db
import pandas as pd
from tqdm import tqdm

#Define the function
def rpdb(gds_file:str = 'Fighting_polygon_array.gds', mesh:str = 'fibo_mesh.csv', target_layer:int = 0, LL:list=(0,0), UR:list=(382, 490),output_file:str = 'Test RDP.gds'):
    #gds is the input gds, and mesh is the matrix of data biasing to be applied, target_layer is the specific layer to be biased
    #LL is the lower left corner of the layer to be edited that contains structures, UR is the upper right corner (in um)
    #Will hardcode these for now for testing purposes

    #Import the .gds
    gds = db.Layout()
    gds.read(gds_file)
    print('File read...')

    #Sets the database units as 1nm
    gds.dbu = 0.001

    #Hard coding layer levels (this can be improved in the future...)
    bias_lyr_index = 42
    final_lyr_index = 43

    #Create bias layer for eventual use
    bias_lyr = gds.layer(bias_lyr_index,0,'Initial Biasing Layer')

    #Create final bias layer
    final_lyr = gds.layer(final_lyr_index,0,'Final Biased Layer')

    #Import the mesh
    pdb = pd.read_csv(mesh, index_col=0)
    print('Mesh read...')

    #Determine mesh columns
    col_len = len(pdb.columns)+1
    columns = range(1,col_len)
    print(f'There are {col_len} columns in the mesh')

    #Determine mesh rows
    row_len = len(pdb)+1
    rows = range(1,row_len)
    print(f'There are {row_len} rows in the mesh')

    #Determine layer sizing
    x_size = UR[0] - LL[0]
    y_size = UR[1] - LL[1]

    #Define size of meshing
    mesh_x = x_size / (len(columns))
    mesh_y = y_size / (len(rows))
    print(f'Mesh X is {mesh_x}nm')
    print(f'Mesh Y is {mesh_y}nm')

    #Calls the top cell (assumes(!) there is one top cell and it contains all the constituent cells)
    top_cell = gds.top_cell()
    print('Top cell chosen, entering loop...')

    #Loop across the mesh, define the regions in the copy layer to copy into the bias layer, then resize the features in the bias layer based on the mesh input
    for i in tqdm(rows):
        for j in tqdm(columns):
                #Define the edges of the mesh at each location
                l_edge = LL[0]+mesh_x*(j-1)
                b_edge = UR[1]-mesh_y*i
                r_edge = LL[0]+mesh_x*j
                t_edge = UR[1]-mesh_y*(i-1)

                #Define the biasing at each location
                new_j = str(j)
                biasing = pdb[new_j][i]

                #Takes polygons from the copy layer in a given search region to keep in the "touching" pya.Region 
                search_region = db.DBox(l_edge,b_edge,r_edge,t_edge)
                touching = db.Region(top_cell.begin_shapes_rec_touching(target_layer, search_region))

                #Biases "touching" and copies it into the top cell in the bias layer
                touch_up = touching.sized(int(biasing))
                top_cell.shapes(bias_lyr).insert(touch_up)
                
    
    print("Finished Mesh Loop")

    #Merges any overlapping polygons and places the final biased data onto the "final" layer
    bias_merge = db.Region(top_cell.begin_shapes_rec_touching(bias_lyr,db.DBox(LL[0],LL[1],UR[0],UR[1])))
    bias_merge.merge()
    top_cell.shapes(final_lyr).insert(bias_merge)

    #Removes unneeded bias layer from the layout
    gds.clear_layer(bias_lyr)
    

    #Writes the new file
    return gds.write(output_file)




rpdb()

File read...
Mesh read...
There are 9 columns in the mesh
There are 9 rows in the mesh
Mesh X is 47.75nm
Mesh Y is 61.25nm
Top cell chosen, entering loop...


100%|██████████| 8/8 [00:06<00:00,  1.15it/s]

Finished Mesh Loop


New Test (w/ clips) (updated 5/13/23)

In [4]:
#Import the needed modules
import klayout.db as db
import pandas as pd
from tqdm import tqdm

#Define the function
def rpdb(gds_file:str = 'Fighting_polygon_array.gds', mesh:str = 'fibo_mesh.csv', target_layer:int = 0, LL:list=(0,0), UR:list=(99500, 99938),output_file:str = 'Test RDP.gds'):
    #gds is the input gds, and mesh is the matrix of data biasing to be applied, target_layer is the specific layer to be biased
    #LL is the lower left corner of the layer to be edited that contains structures, UR is the upper right corner (in um)
    #Will hardcode these for now for testing purposes

    #Actual extents for test: 99950, 99938

    #Import the .gds
    gds = db.Layout()
    gds.read(gds_file)
    print(f'File read: {gds_file}')

    #Sets the database units as 1nm
    gds.dbu = 0.001

    #Hard coding layer levels (this can be improved in the future...)
    bias_lyr_index = 49
    
    #Create bias layer for eventual use
    bias_lyr = gds.layer(bias_lyr_index,0,'Initial Biasing Layer')

    #Import the mesh
    pdb = pd.read_csv(mesh, index_col=0)
    print(f'Mesh read: {mesh}')

    #Determine maximum absolute bias applied, in order to size up the cell regions
    mesh_max = pdb.max().max()
    mesh_min = pdb.min().min()
    if abs(mesh_max) >= abs(mesh_min):
         bias_max = mesh_max
    else:
         bias_max = abs(mesh_min)

    #Determine mesh columns
    col_len = len(pdb.columns)+1
    columns = range(1,col_len)
    print(f'There are {col_len} columns in the mesh')

    #Determine mesh rows
    row_len = len(pdb)+1
    rows = range(1,row_len)
    print(f'There are {row_len} rows in the mesh')

    #Determine layer sizing
    x_size = UR[0] - LL[0]
    y_size = UR[1] - LL[1]

    #Define size of meshing
    mesh_x = x_size / (len(columns))
    mesh_y = y_size / (len(rows))
    print(f'Mesh X is {mesh_x}nm')
    print(f'Mesh Y is {mesh_y}nm')

    #Calls the top cell (assumes(!) there is one top cell and it contains all the constituent cells) and stores index of all called cells
    top_cell = gds.top_cell()
    top_cell_index = top_cell.cell_index()
    called_cells = top_cell.called_cells()
    print('Top cell chosen, entering loop...')

    #Loop across the mesh, define the regions in the copy layer to copy into the bias layer, then resize the features in the bias layer based on the mesh input
    for i in tqdm(rows):
        for j in columns:
                #Define the edges of the mesh at each location
                l_edge = LL[0]+(mesh_x*(j-1)-bias_max)
                b_edge = UR[1]-(mesh_y*i+bias_max)
                r_edge = LL[0]+(mesh_x*j+bias_max)
                t_edge = UR[1]-(mesh_y*(i-1)-bias_max)

                #Define the biasing at each location
                new_j = str(j)
                biasing = pdb[new_j][i]
                #print(f'This is the bias value: {biasing}')

                #Clips mesh area from the polygon, preserving hierarchy, and names it
                clip = gds.clip(top_cell,db.DBox(l_edge,b_edge,r_edge,t_edge))
                clip.name= f'Mesh_Cell_{i}_{j}'
                

                #Loop to identify each shape and size it up by the bias value amount
                #print(f'Clip called cells are: {clip.called_cells()}')

                #Copies the cell tree from the clip into a new bias cell
                bias_cell = gds.create_cell(f'bias_cell_{i}_{j}')
                bias_tree = bias_cell.copy_tree(clip)
                #print(f'Biasing is {biasing}')

                #Goes through the hierarchy of each clip and biases by the appropriate value, and inserts into the bias layer
                for k in bias_cell.called_cells():
                    copy_cell = gds.cell(k)
                    search_region = db.Region(copy_cell.bbox(target_layer))
                    shapes_region = db.Region(copy_cell.shapes(target_layer))
                    touching = search_region & shapes_region
                    bias_apply = touching.sized(int(biasing))
                    copy_cell.shapes(bias_lyr).insert(bias_apply)

                                    
                clip.prune_cell()                    
    
    print("Finished Mesh Loop")
    
    #Writes the new file
    return gds.write(output_file)

rpdb()

File read: Fighting_polygon_array.gds
Mesh read: fibo_mesh.csv
There are 9 columns in the mesh
There are 9 rows in the mesh
Mesh X is 12437.5nm
Mesh Y is 12492.25nm
Top cell chosen, entering loop...


100%|██████████| 8/8 [00:15<00:00,  1.91s/it]


Finished Mesh Loop
